# Home Exercise on Language Modeling
Implement a transformer to sovle Language Modeling task.

Data: wikitext-103 
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-103-v1")
Note: Submit only a single Jupyter Notebook file that can handle all tasks, including data downloading, preprocessing, model training and model evaluation. (Submissions that do not follow the guidelines will receive a score of 0.)

# Step 1: Install libraries and import

In [1]:
# --- 1. Install libraries if needed ---
!pip install datasets transformers torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible

In [2]:
# --- 2. Import libraries ---
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset
import math
from tqdm import tqdm

# Step 2: Load the dataset from huggingface

In [3]:
# --- 3. Load the dataset ---
dataset = load_dataset("wikitext", "wikitext-103-v1")
print(dataset)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


# Step 3: Preprocessing data

In [4]:
# --- 4. Preprocessing: Tokenization ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # GPT-2 tokenizer (simple and effective)
tokenizer.pad_token = tokenizer.eos_token  # Avoid pad token issues

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
block_size = 128

# 5. Tokenize
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=block_size,
        return_special_tokens_mask=True
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
# 6. Group
def group_texts(examples):
    return {
        "input_ids": examples["input_ids"],
        "labels": examples["input_ids"],
        "attention_mask": examples["attention_mask"],
    }

lm_datasets = tokenized_datasets.map(group_texts, batched=True)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
# --- 7. Create DataLoaders ---
train_dataset = lm_datasets["train"]
test_dataset = lm_datasets["test"]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Step 4: Building the model

In [8]:
# --- 8. Define a simple Transformer model for Language Modeling ---
class TransformerLM(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=6, dim_feedforward=2048):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Parameter(torch.zeros(1, 1024, d_model))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward),
            num_layers
        )
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        src = self.embedding(src) + self.pos_encoder[:, :src.size(1), :]
        src = src.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        output = self.transformer(src)
        output = output.permute(1, 0, 2)  # (batch_size, seq_len, d_model)
        logits = self.fc_out(output)
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerLM(vocab_size=len(tokenizer)).to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [9]:
# --- 9. Training setup ---
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Step 5: Training the model

In [10]:
# --- 10. Training loop ---
epochs = 2

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        input_ids = torch.stack(batch["input_ids"]).to(device)
        labels = torch.stack(batch["labels"]).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} Average Loss: {total_loss/len(train_loader)}")

Epoch 1: 100%|██████████| 56293/56293 [3:32:26<00:00,  4.42it/s, loss=3.29]


Epoch 1 Average Loss: 3.157044642708471


Epoch 2: 100%|██████████| 56293/56293 [3:32:43<00:00,  4.41it/s, loss=4.41]

Epoch 2 Average Loss: 3.4097918827098797


# Step 6: Evaluating the model

In [11]:
# --- 11. Evaluation (Perplexity) ---
model.eval()
total_loss = 0
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = torch.stack(batch["input_ids"]).to(device)
        labels = torch.stack(batch["labels"]).to(device)

        outputs = model(input_ids)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        total_loss += loss.item()

average_loss = total_loss / len(test_loader)
perplexity = math.exp(average_loss)
print(f"Test Perplexity: {perplexity:.2f}")

100%|██████████| 137/137 [00:10<00:00, 13.06it/s]

Test Perplexity: 29.88


In [12]:
# --- Evaluation Example: Text Generation ---
import torch.nn.functional as F

def generate_text(model, tokenizer, prompt, max_new_tokens=50, top_k=50):
    model.eval()
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    generated = input_ids
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(generated)
            next_token_logits = outputs[:, -1, :]

            # Top-k sampling
            values, indices = torch.topk(next_token_logits, top_k)
            probs = F.softmax(values, dim=-1)
            next_token = indices[0, torch.multinomial(probs[0], num_samples=1)]

            next_token = next_token.unsqueeze(0)  # (1, 1)
            generated = torch.cat((generated, next_token), dim=1)

    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    return generated_text

# --- Example prompts ---
example_prompts = [
    "The meaning of life is",
    "Artificial intelligence is transforming",
    "The history of France began",
]

# --- Generate texts ---
for prompt in example_prompts:
    print(f"\n=== Prompt: {prompt} ===")
    output_text = generate_text(model, tokenizer, prompt, max_new_tokens=30)
    print(output_text)



=== Prompt: The meaning of life is ===
The meaning of life is = at and in is of for of

=== Prompt: Artificial intelligence is transforming ===
Artificial intelligence is transforming in the ' for < = to and

=== Prompt: The history of France began ===
The history of France began = . as the- in = the
